In [32]:
install.packages("tidyverse")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [33]:
library(tidyverse)
library(dplyr)

In [34]:
data <- read_csv("train.csv")
head(data)

New names:
• `` -> `...1`
Rows: 5847 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): Name, Location, Fuel_Type, Transmission, Owner_Type, Mileage, Engin...
dbl (5): ...1, Year, Kilometers_Driven, Seats, Price

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


...1,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5,NA,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5,8.61 Lakh,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7,NA,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5,NA,17.74
6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5,NA,3.50
7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36 kmpl,2755 CC,171.5 bhp,8,21 Lakh,17.50


#b)  Removing Units from Columns

In [35]:
# Remove units and convert to numeric
data$Mileage <- as.numeric(gsub(" kmpl| km/kg", "", data$Mileage))
data$Engine <- as.numeric(gsub(" CC", "", data$Engine))
data$Power <- as.numeric(gsub(" bhp", "", data$Power))
head(data)

...1,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.20,5,NA,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13.00,1199,88.70,5,8.61 Lakh,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7,NA,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968,140.80,5,NA,17.74
6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08,1461,63.10,5,NA,3.50
7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36,2755,171.50,8,21 Lakh,17.50


#a) Checking for Missing values, either impute or drop them and justify the action

In [36]:
# Check for missing values in each column
colSums(is.na(data))


...1              Name          Location              Year 
                0                 0                 0                 0 
Kilometers_Driven         Fuel_Type      Transmission        Owner_Type 
                0                 0                 0                 0 
          Mileage            Engine             Power             Seats 
                2                36                36                38 
        New_Price             Price 
             5032                 0

**New_Price** has over 86% missing values, making it challenging to impute accurately without bias. Given the limited availability of data here, dropping this column is the most practical approach .

**Mileage, Engine, Power, and Seats:**
These columns have minimal missing values (all under 1% of total data).
Since these are essential features for vehicle specifications, it’s more reasonable to impute missing values rather than discard them.
replacing with mode for categorical-like columns (Seats which has a limited set of values) and median for numeric columns (Mileage, Engine, Power). Median is more robust to outliers than the mean, making it a good choice for these features.


In [37]:
data$New_Price <- NULL
head(data)

...1,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.20,5,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13.00,1199,88.70,5,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968,140.80,5,17.74
6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08,1461,63.10,5,3.50
7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36,2755,171.50,8,17.50


In [38]:
# Define a custom Mode function
Mode <- function(x) {
  ux <- unique(x)
  ux[which.max(tabulate(match(x, ux)))]
}

# Impute missing values
data$Mileage[is.na(data$Mileage)] <- median(data$Mileage, na.rm = TRUE)
data$Engine[is.na(data$Engine)] <- median(data$Engine, na.rm = TRUE)
data$Power[is.na(data$Power)] <- median(data$Power, na.rm = TRUE)
data$Seats[is.na(data$Seats)] <- Mode(data$Seats)

colSums(is.na(data))

...1              Name          Location              Year 
                0                 0                 0                 0 
Kilometers_Driven         Fuel_Type      Transmission        Owner_Type 
                0                 0                 0                 0 
          Mileage            Engine             Power             Seats 
                0                 0                 0                 0 
            Price 
                0

#c) One-hot encoding

In [39]:
# Convert categorical variables to one-hot encoded columns
data_cleaned <- cbind(data, model.matrix(~ Fuel_Type + Transmission-1, data = data))
# Remove original categorical columns
data_cleaned$Fuel_Type <- NULL
data_cleaned$Transmission <- NULL
head(data_cleaned)

,...1,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_TypeDiesel,Fuel_TypeElectric,Fuel_TypePetrol,TransmissionManual
,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582,126.20,5,12.50,1,0,0,1
2,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199,88.70,5,4.50,0,0,1,1
3,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248,88.76,7,6.00,1,0,0,1
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968,140.80,5,17.74,1,0,0,0
5,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461,63.10,5,3.50,1,0,0,1
6,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,First,11.36,2755,171.50,8,17.50,1,0,0,0


# d)Creating new column using mutate


In [40]:
# Creating Kilometers_per_Year feature using mutate
data_cleaned <- data_cleaned %>%
  mutate(Kilometers_per_Year = Kilometers_Driven / (2024 - Year))
head(data_cleaned)


,...1,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_TypeDiesel,Fuel_TypeElectric,Fuel_TypePetrol,TransmissionManual,Kilometers_per_Year
,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582,126.20,5,12.50,1,0,0,1,4555.556
2,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199,88.70,5,4.50,0,0,1,1,3538.462
3,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248,88.76,7,6.00,1,0,0,1,7250.000
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968,140.80,5,17.74,1,0,0,0,3697.273
5,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461,63.10,5,3.50,1,0,0,1,7909.000
6,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,First,11.36,2755,171.50,8,17.50,1,0,0,0,4500.000


# e) select, filter, rename, mutate, arrange and summarize with group by operations

In [41]:
# Select specific columns
selected_data <- data %>%
  select(Name, Location, Year, Kilometers_Driven, Price)

head(selected_data)


Name,Location,Year,Kilometers_Driven,Price
<chr>,<chr>,<dbl>,<dbl>,<dbl>
Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,12.50
Honda Jazz V,Chennai,2011,46000,4.50
Maruti Ertiga VDI,Chennai,2012,87000,6.00
Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,17.74
Nissan Micra Diesel XV,Jaipur,2013,86999,3.50
Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,17.50


In [42]:
# Filter rows where the price is greater than a certain amount
filtered_data <- data %>%
  filter(Price > 10)

head(filtered_data)

...1,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.20,5,12.50
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968,140.80,5,17.74
7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36,2755,171.50,8,17.50
13,Land Rover Range Rover 2.2L Pure,Delhi,2014,72000,Diesel,Automatic,First,12.70,2179,187.70,5,27.00
14,Land Rover Freelander 2 TD4 SE,Pune,2012,85000,Diesel,Automatic,Second,0.00,2179,115.00,5,17.50
15,Mitsubishi Pajero Sport 4X4,Delhi,2014,110000,Diesel,Manual,First,13.50,2477,175.56,7,15.00


In [43]:
# Rename columns for clarity
renamed_data <- data %>%
  rename(Engine_Power = Power, Car_Year = Year)
head(renamed_data)

...1,Name,Location,Car_Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Engine_Power,Seats,Price
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.20,5,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13.00,1199,88.70,5,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968,140.80,5,17.74
6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08,1461,63.10,5,3.50
7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36,2755,171.50,8,17.50


In [44]:
# Add a new column 'car_age'
mutated_data <- data %>%
  mutate(Car_Age = (2024 - Year))
head(mutated_data)

...1,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Car_Age
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.20,5,12.50,9
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13.00,1199,88.70,5,4.50,13
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7,6.00,12
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968,140.80,5,17.74,11
6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08,1461,63.10,5,3.50,11
7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36,2755,171.50,8,17.50,8


In [45]:
# Arrange data by Price in descending order
arranged_data <- data %>%
  arrange(desc(Price))
head(arranged_data)

...1,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4079,Land Rover Range Rover 3.0 Diesel LWB Vogue,Hyderabad,2017,25000,Diesel,Automatic,First,13.33,2993,255.0,5,160.00
5781,Lamborghini Gallardo Coupe,Delhi,2011,6500,Petrol,Automatic,Third,6.40,5204,560.0,2,120.00
5919,Jaguar F Type 5.0 V8 S,Hyderabad,2015,8000,Petrol,Automatic,First,12.50,5000,488.1,2,100.00
1505,Land Rover Range Rover Sport SE,Kochi,2019,26013,Diesel,Automatic,First,12.65,2993,255.0,5,97.07
1974,BMW 7 Series 740Li,Coimbatore,2018,28060,Petrol,Automatic,First,12.05,2979,320.0,5,93.67
1984,BMW 7 Series 740Li,Bangalore,2017,17465,Petrol,Automatic,First,12.05,2979,320.0,5,93.00


In [46]:
# Summarize average price by Fuel Type
summary_data <- data %>%
  group_by(Fuel_Type) %>%
  summarize(Average_Price = mean(Price, na.rm = TRUE),
            Total_Cars = n())

head(summary_data)

Fuel_Type,Average_Price,Total_Cars
<chr>,<dbl>,<int>
Diesel,12.960686,3161
Electric,12.875000,2
Petrol,5.756688,2684
